In [45]:
import os
import requests
import datetime
# import calendar

import pandas as pd
from dotenv import load_dotenv
from sqlmodel import SQLModel, create_engine
from sqlalchemy.orm import sessionmaker

from heatmap.conversion import ng_to_core_power_carbon_intensity
from heatmap.definitions.ng_carbon_intensity import Response as NgCarbonIntensityResponse
from heatmap.definitions.timeseries import PowerCarbonIntensity, PowerPrice, GasPrice

In [46]:
load_dotenv('../.env')
DB_CONN_STR = os.environ['DB_CONN_STR'].replace('sqlite:///', 'sqlite:///../')

engine = create_engine(url=DB_CONN_STR)
bound_sessionmaker = sessionmaker(engine)
SQLModel.metadata.create_all(engine, tables=[
    PowerCarbonIntensity.__table__, 
    PowerPrice.__table__, 
    GasPrice.__table__
])

In [39]:
def batch_start_end_dates(start_date: datetime.datetime, end_date: datetime.datetime, freq: str = '7d') -> list[tuple[datetime.datetime, datetime.datetime]]:
    dt_rng = pd.date_range(start_date, end_date, freq=freq)
    return list(zip([dt+datetime.timedelta(minutes=30) for dt in dt_rng[:-1]], dt_rng[1:]))

In [43]:
start_date = datetime.datetime(2021, 1, 1, 0, 0)
end_date = datetime.datetime(2021, 3, 30, 23, 59)

power_carbon_intensities = []

for batch_start_date, batch_end_date in batch_start_end_dates(start_date, end_date):
    r = requests.get(f'https://api.carbonintensity.org.uk/intensity/{batch_start_date.strftime("%Y-%m-%dT%H:%MZ")}/{batch_end_date.strftime("%Y-%m-%dT%H:%MZ")}', headers={'Accept': 'application/json'})
    ng_carbon_intensity_response = NgCarbonIntensityResponse.model_validate(r.json())
    power_carbon_intensities.extend(ng_to_core_power_carbon_intensity(ng_carbon_intensity_response))

with bound_sessionmaker() as session:
    session.bulk_save_objects(power_carbon_intensities)
    session.commit()

power_carbon_intensities[-5:]

[PowerCarbonIntensity(start_time=datetime.datetime(2021, 3, 25, 21, 30, tzinfo=TzInfo(UTC)), end_time=datetime.datetime(2021, 3, 25, 22, 0, tzinfo=TzInfo(UTC)), value=85.0),
 PowerCarbonIntensity(start_time=datetime.datetime(2021, 3, 25, 22, 0, tzinfo=TzInfo(UTC)), end_time=datetime.datetime(2021, 3, 25, 22, 30, tzinfo=TzInfo(UTC)), value=82.0),
 PowerCarbonIntensity(start_time=datetime.datetime(2021, 3, 25, 22, 30, tzinfo=TzInfo(UTC)), end_time=datetime.datetime(2021, 3, 25, 23, 0, tzinfo=TzInfo(UTC)), value=79.0),
 PowerCarbonIntensity(start_time=datetime.datetime(2021, 3, 25, 23, 0, tzinfo=TzInfo(UTC)), end_time=datetime.datetime(2021, 3, 25, 23, 30, tzinfo=TzInfo(UTC)), value=73.0),
 PowerCarbonIntensity(start_time=datetime.datetime(2021, 3, 25, 23, 30, tzinfo=TzInfo(UTC)), end_time=datetime.datetime(2021, 3, 26, 0, 0, tzinfo=TzInfo(UTC)), value=74.0)]

In [41]:
r = requests.get(f'https://api.carbonintensity.org.uk/intensity/{batch_start_date.strftime("%Y-%m-%dT%H:%MZ")}/{batch_end_date.strftime("%Y-%m-%dT%H:%MZ")}', headers={'Accept': 'application/json'})
 
ng_carbon_intensity_response = NgCarbonIntensityResponse.model_validate(r.json())
power_carbon_intensities = ng_to_core_power_carbon_intensity(ng_carbon_intensity_response)

print(len(power_carbon_intensities))
power_carbon_intensities[:5]

336


[PowerCarbonIntensity(start_time=datetime.datetime(2021, 3, 19, 0, 0, tzinfo=TzInfo(UTC)), end_time=datetime.datetime(2021, 3, 19, 0, 30, tzinfo=TzInfo(UTC)), value=171.0),
 PowerCarbonIntensity(start_time=datetime.datetime(2021, 3, 19, 0, 30, tzinfo=TzInfo(UTC)), end_time=datetime.datetime(2021, 3, 19, 1, 0, tzinfo=TzInfo(UTC)), value=170.0),
 PowerCarbonIntensity(start_time=datetime.datetime(2021, 3, 19, 1, 0, tzinfo=TzInfo(UTC)), end_time=datetime.datetime(2021, 3, 19, 1, 30, tzinfo=TzInfo(UTC)), value=167.0),
 PowerCarbonIntensity(start_time=datetime.datetime(2021, 3, 19, 1, 30, tzinfo=TzInfo(UTC)), end_time=datetime.datetime(2021, 3, 19, 2, 0, tzinfo=TzInfo(UTC)), value=169.0),
 PowerCarbonIntensity(start_time=datetime.datetime(2021, 3, 19, 2, 0, tzinfo=TzInfo(UTC)), end_time=datetime.datetime(2021, 3, 19, 2, 30, tzinfo=TzInfo(UTC)), value=167.0)]

In [37]:
power_carbon_intensities[-5:]

[PowerCarbonIntensity(start_time=datetime.datetime(2021, 3, 25, 21, 30, tzinfo=TzInfo(UTC)), end_time=datetime.datetime(2021, 3, 25, 22, 0, tzinfo=TzInfo(UTC)), value=85.0),
 PowerCarbonIntensity(start_time=datetime.datetime(2021, 3, 25, 22, 0, tzinfo=TzInfo(UTC)), end_time=datetime.datetime(2021, 3, 25, 22, 30, tzinfo=TzInfo(UTC)), value=82.0),
 PowerCarbonIntensity(start_time=datetime.datetime(2021, 3, 25, 22, 30, tzinfo=TzInfo(UTC)), end_time=datetime.datetime(2021, 3, 25, 23, 0, tzinfo=TzInfo(UTC)), value=79.0),
 PowerCarbonIntensity(start_time=datetime.datetime(2021, 3, 25, 23, 0, tzinfo=TzInfo(UTC)), end_time=datetime.datetime(2021, 3, 25, 23, 30, tzinfo=TzInfo(UTC)), value=73.0),
 PowerCarbonIntensity(start_time=datetime.datetime(2021, 3, 25, 23, 30, tzinfo=TzInfo(UTC)), end_time=datetime.datetime(2021, 3, 26, 0, 0, tzinfo=TzInfo(UTC)), value=74.0)]

In [38]:
bound_sessionmaker = sessionmaker(engine)

with bound_sessionmaker() as session:
    for power_carbon_intensity in power_carbon_intensities:
        session.add(power_carbon_intensity)
    session.commit()